### Skip-Gram 모델 구현

In [1]:
from mynlp import preprocess, create_contexts_target, convert_one_hot
from nn_layers import MatMul, SoftmaxWithLoss, Adam, SGD, Trainer
import numpy as np

# contexts와 target을 구하는 함수 : mynlp.py에 추가해 놓는다
# convert_one_hot 원핫 변환 함수 : mynlp.py 에 추가해 놓는다
# Trainer 클래스  : nn_layers에 추가해 놓는다

In [ ]:
class SimpleSkipGram:
    def __init__(self,vocab_size, hidden_size): # vocab_size=7,hidden_size5
        V, H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01*np.random.randn(V,H).astype('f') # (7,5)
        W_out = 0.01*np.random.randn(H,V).astype('f') # (5,7)
        
        # 계층 생성
        self.in_layer = MatMul(W_in)         # (6,7)*(7,5) = (6,5)
        self.out_layer = MatMul(W_out)       # (6,5)*(5,7) = (6,7)
        self.loss_layer1 = SoftmaxWithLoss()
        self.loss_layer2 = SoftmaxWithLoss()

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer, self.out_layer]
        self.params, self.grads = [],[]
        for layer in layers : # 2회
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수 단어의 분산 표현을 저장한다.    
        self.word_vec = W_in 
        
    def predict(self, target):  # target: 입력값
        h = self.in_layer.forward(target)
        s = self.out_layer.forward(h)
        return s
        
    def forward(self, contexts, target):  # contexts : 실제값  (6,2,7) 3차원, target: 입력값
        h = self.in_layer.forward(target)  # (6,5)
        s = self.out_layer.forward(h)      # (6,7)
        l1 = self.loss_layer1.forward(s,contexts[:,0])
        l2 = self.loss_layer2.forward(s,contexts[:,1])
        loss = l1 + l2
        return loss
            
        